In [1]:
import pandas as pd
import glob


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from pathlib import Path

In [2]:
input_files = str(Path('../4_szures/data/*.csv'))

In [3]:
x = pd.read_csv(glob.glob(input_files)[0]).drop(["Unnamed: 0","Unnamed: 0.1"],axis=1)\
.pipe(lambda df:df.loc[df["sold"]==1]).fillna(0)

In [4]:
x = x.loc[x["vegso_ar"]!=0]


In [5]:
y = x["vegso_ar"]
lg10_y = pd.np.log10(y)
x["lg10_AR"] = lg10_y

x = x.drop(['vegso_ar',"lg10_AR"], axis=1)


In [6]:
szukitett = x[["hany_eves","hany_eves_2","MEGTETT","MEGTETT_2","KLÍMA","LÉGZSÁK",\
               "ALUFELNI","ASR","AUTOMATA","BI-XENON","BLUETOOTH","TELJESITMENY LOERO",\
               "audi","mercedes-benz","bmw","infiniti","jaguar","land_rover","porsche",\
               "lexus","maserati","Hibrid","Elektromos","Dízel","Benzin"]].fillna(0)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(szukitett.dropna(), lg10_y, test_size=0.2, random_state=42)

lreg = LinearRegression()

result = lreg.fit(x_train, y_train)

y_test_pred = result.predict(x_test)

lin_mse = mean_squared_error(y_test, y_test_pred)
lin_rmse = pd.np.sqrt(lin_mse)
lin_rmse

print(result.intercept_)


6.587655503948437


In [8]:
pd.DataFrame(szukitett.loc[9]).to_csv("proba.csv")

In [9]:
import pickle


output_file = str(Path('data/finalized_model.sav'))

# save the model to disk

pickle.dump(result, open(output_file, 'wb'))

In [107]:
import json

In [41]:
szukitett["hany_eves"].nunique()

41

In [44]:
nuniques = [szukitett[col].nunique() for col in szukitett.columns]

In [57]:
number_vals = [idx for idx,val in enumerate(nuniques) if val >2]

In [64]:
auto_marka = ['audi', 'mercedes-benz', 'bmw', 'infiniti',
              'jaguar', 'land_rover', 'porsche', 'lexus', 'maserati']
hajtaslanc = ['Hibrid', 'Elektromos', 'Dízel', 'Benzin']
felszereltseg = szukitett.columns.drop(auto_marka + hajtaslanc + list(szukitett.columns[number_vals]))

In [65]:
felszereltseg

Index(['KLÍMA', 'LÉGZSÁK', 'ALUFELNI', 'ASR', 'AUTOMATA', 'BI-XENON',
       'BLUETOOTH'],
      dtype='object')

In [116]:
details = {"number":list(szukitett.columns[number_vals]),"Dropdown":{"auto_marka":auto_marka,"hajtaslanc":hajtaslanc}\
          ,"boolen":list(felszereltseg)}

In [117]:
json_file = json.dumps(details)
f = open("dict.json","w")
f.write(json_file)
f.close()

In [122]:
with open('dict.json') as json_file:  
    data = json.load(json_file)

In [27]:
result.coef_

array([-6.67337663e-02,  1.02209617e-03, -5.59380994e-07,  6.60755176e-14,
        2.17679007e-02, -1.34991729e-02,  4.70939703e-02,  2.42267445e-02,
        5.40959252e-02,  2.20176724e-02,  1.16100102e-02,  1.98509733e-03,
        9.77286415e-02,  1.22520958e-01,  8.45762947e-02,  3.99166686e-02,
        4.54006378e-02,  2.05456232e-01,  2.29013232e-01,  1.42573703e-01,
        1.23437925e-01,  3.20587568e-02,  3.66179902e-01,  9.30756745e-04,
       -5.09140080e-02])

In [31]:
len(x_train.columns)

25